# Utility code for identifying primes within an interval
This is very specific code for identifying arrays of consecutive primes within an interval.
The interval is defined by its starting point $x_0$ and its length 2*<i>blocklength</i>.  Since our algorithm only
searches over odd numbers, the parameter <i>blocklength</i> effects a search from $x_0$ to $x_0 + 2\cdot {\rm blocklength}$.

The search is performed by the function <i>pblock11()</i>.  This is Eratosthenes sieve with a couple of accelerations.  We search
only over odd numbers, each odd number is represented by a single bit in a boolean array, and we initialize the array with the
pattern from ${\mathcal G}(11^\#)$.  So the iterations start from $p=13$.

Our search is conducted by eliminating multiples of the primes in the array <i>smallprimes[]</i>.  So the range of our search
wanes at <i>smallprimes[-1]**2</i>, the square of the largest prime in this array.

In [1]:
import numpy as np
import array
import gc
import psutil
import sys
import os

In [2]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 3340.66 MB


In [3]:
# create the binary pattern for 11-rough numbers - note - we only look at the odd numbers
try:
    G11 = np.load("G11uint.npy")
except FileNotFoundError:
    print("File G11uint.npy not found!")
    sys.exit()
    
rough11 = np.zeros(1155, dtype=bool)
i=0
j=0
while (i < 1155):
    rough11[i]=True
    i += int(G11[j]/2)
    j += 1

rough11[-100:]

array([ True,  True, False,  True,  True, False, False, False, False,
        True,  True, False, False,  True, False,  True,  True, False,
        True, False, False,  True, False, False,  True,  True, False,
       False, False, False,  True,  True, False, False,  True, False,
        True, False, False, False, False, False, False,  True, False,
        True,  True, False,  True,  True, False,  True, False, False,
       False,  True, False, False,  True, False,  True, False, False,
        True,  True, False,  True, False, False,  True,  True, False,
       False,  True, False, False,  True, False,  True,  True, False,
        True, False, False,  True,  True, False, False,  True, False,
        True,  True, False,  True, False, False, False, False, False,
        True])

In [4]:
# identify the primes among odd numbers in a block of size blocklength starting at x0
# this version repeats the pattern from G(11#) to accelerate execution.
# 
def pblock11(x0, blocklength):
    maxp = np.floor(np.sqrt(x0+2*blocklength))
    if (maxp > smallprimes[-1]):
        print(f"ERROR: need prime {maxp} beyond available list {smallprimes[-1]}")
        return

    if (x0 <= maxp):  # the starting point is within the range of primes needed in the calculation
        xx0 = maxp
    else:
        xx0 = x0

    if ((xx0 % 2) == 0):  # our starting point xx0 must be odd
        xx0 = xx0+1

    # value associated to index i is xx0+2i
    # initialize the array with the pattern from G(11#)
    # 
    res0 = int(xx0 % 2310)
    i = int((res0 - 1)/2)  # map value to index in boolean block over odd numbers
    ncopies = int(np.ceil(blocklength / 1155))
    adj_blocklen = ncopies * 1155

    # if needed, adjust the maximum prime that we need to check
    maxp1 = np.floor(np.sqrt(xx0 + 2*adj_blocklen))
    if maxp1 > maxp:
        print(f"Adjusted maxp {maxp} {blocklength} to {maxp1} {adj_blocklen}")
        maxp = maxp1
    
    if (i == 0):
        is_prime = np.tile(rough11, ncopies)
    else:
        shifted_rough11 = np.concatenate((rough11[i:],rough11[0:i]))
        is_prime = np.tile(shifted_rough11, ncopies)

#    is_prime = np.ones(blocklength,dtype=bool)
    print(f"res0 {res0} i {i} x0 {xx0} block {len(is_prime)} pbound {maxp}")

    i=4 # index for smallprimes, start at p=13
    j=0 # index for testlength
    p = smallprimes[i]
    if (p != 13):
        print(f"i {i} prime {p} should be 13")
        return
        
    while (p < maxp):
        if (p < 3000000):
            print(f"checking {p} of {maxp}", end='\r')
        elif ( (i % 1024) == 0):
            print(f"{i} checking {p} of {maxp}", end='\r')
            
        res0 = xx0 % p  # starting residue
        delta0 = int(p - res0)
        if (res0 == 0):
            j = 0
        elif ((delta0 % 2) ==0):
            j = int(delta0 /2)
        else:
            j = int((p+delta0)/2)
        while (j < adj_blocklen):
            is_prime[j] = False
            j += p
        i += 1
        p = smallprimes[i]

    pdexes = np.array(np.where(is_prime))
    pdexes.shape

    newprimes = (2*pdexes[0]+xx0)
    print()
    print(f"at {i} {len(newprimes)} primes from {newprimes[0]} to {newprimes[-1]}")
    return newprimes

In [5]:
# generate the small primes that we need for creating sequences of consecutive primes among large numbers
# Our searches are limited by p^2 for the largest of the small primes p.
# Establish the small primes that we need, up over a billion - 
if os.path.exists('primesE9.npy'):
    primesE9 = np.load('primesE9.npy')
    print(f"primesE9 {len(primesE9)} primes from {primesE9[0]} to {primesE9[-1]}")
    smallprimes = primesE9
elif os.path.exists('primes23.npy'):
    primes23 = np.load('primes23.npy')
    print(f"primes23 {len(primes23)} primes from {primes23[0]} to {primes23[-2]}")
    smallprimes = np.concatenate(([3,5,7,11,13,17,19,23],primes23[:-1]))
    # Create primesE9
    x0 = smallprimes[-2]+2
    ext_length = 400001000
    prime_extension = pblock11(x0,ext_length)
    # store primesE9 to file for future use
    smallprimes = np.concatenate((smallprimes[:-1],prime_extension))
    np.save('primesE9.npy', smallprimes)
elif os.path.exists('primes19.npy'):
    primes19 = np.load('primes19.npy')
    print(f"primes19 {len(primes19)} primes from {primes19[0]} to {primes19[-2]}")
    smallprimes = np.concatenate(([3,5,7,11,13,17,19],primes19[:-1]))
    # Create primesE9
    x0 = smallprimes[-2]+2
    ext_length = 506700000
    prime_extension = pblock11(x0,ext_length)
    # store primesE9 to file for future use
    smallprimes = np.concatenate((smallprimes[:-1],prime_extension))
    np.save('primesE9.npy', smallprimes)
else:
    print("ERROR:  we need at least primes19.npy to start")
    sys.exit()

primesE9 51961884 primes from 3 to 1023101273


In [6]:
print(f"{len(smallprimes)} primes from {smallprimes[0]} to {smallprimes[-1]}")

51961884 primes from 3 to 1023101273


In [7]:
smallprimes[23713133]

np.int64(447216097)

## Example usage of pblock11()
We set a starting point $x_0$ for the search.  $x_0$ should be a large odd integer.
The requested interval over which we search begins at $x_0$ and runs 2*<i>blocklength</i>.  We invoke <i>pblock11(x0, blocklength)</i>
with the parameter <i>blocklength</i>, and since we only search over odd numbers

In [71]:
# test cell
# we want to focus on intervals of survival, so start at a prime-square and allow for the square falling in a large gap
# x0 = smallprimes[-200]**2 - 1000  
# x0 = smallprimes[-8]**2 - 600  # this value starts a gap of 2
# x0 = smallprimes[1496935]**2-600  # this value starts a constellation of 2 4
# x0 = smallprimes[23713133]**2-600  # this value starts a constellation of 4 2
x0 = 800001**2-500
testlength = 900000000
print(f"Searching for primes starting at {x0}")
ptrial = pblock11(x0,testlength)

Searching for primes starting at 640001599501
res0 1111 i 555 x0 640001599501 block 900000255 pbound 801125.0
checking 801107 of 801125.0
at 64045 66214780 primes from 640001599543 to 641801599973


In [72]:
# visually check the result...
print(f"from E{np.log10(ptrial[0]):8f} to E{np.log10(ptrial[-1]):8f}")
print(ptrial[0:30])
print()
print(ptrial[-30:])


from E11.806181 to E11.807401
[640001599543 640001599571 640001599591 640001599607 640001599663
 640001599697 640001599703 640001599729 640001599747 640001599777
 640001599781 640001599801 640001599843 640001599847 640001599883
 640001599903 640001599969 640001600023 640001600057 640001600081
 640001600089 640001600113 640001600159 640001600221 640001600237
 640001600263 640001600327 640001600333 640001600377 640001600389]

[641801599379 641801599429 641801599457 641801599459 641801599463
 641801599501 641801599513 641801599519 641801599529 641801599547
 641801599597 641801599603 641801599627 641801599631 641801599643
 641801599667 641801599687 641801599723 641801599729 641801599781
 641801599787 641801599793 641801599847 641801599853 641801599877
 641801599891 641801599909 641801599939 641801599961 641801599973]


### Extending a block
The function <i>extendblock()</i> takes an array of primes as input and appends the block of primes that follows

In [8]:
def extendblock(prefix_block, blocklength):
    # for large samples 
    x0 = prefix_block[-1]+2
    extension_block = pblock11(x0, blocklength)
    primecombined = np.concatenate((prefix_block,extension_block))
    print(f"prefix length {len(prefix_block)} min {np.sqrt(prefix_block[0]):.2f}:{prefix_block[0]} max {np.sqrt(prefix_block[-1]):.2f}:{prefix_block[-1]}")
    print(f"extension length {len(extension_block)} min {np.sqrt(extension_block[0]):.2f}:{extension_block[0]} max {np.sqrt(extension_block[-1]):.2f}:{extension_block[-1]}")
    print(f"combined length {len(primecombined)} min {np.sqrt(primecombined[0]):.2f}:{primecombined[0]} max {np.sqrt(primecombined[-1]):.2f}:{primecombined[-1]}")

    return primecombined

In [73]:
# we comment out the save commands to protect existing files
# np.save('primeblockE11_640.npy',ptrial)

In [9]:
# Using extendblock() to increase the array of small primes
len(smallprimes), smallprimes[-1]

(51961884, np.int64(1023101273))

In [10]:
smallprimesB = extendblock(smallprimes, 900000000)

res0 2275 i 1137 x0 1023101275 block 900000255 pbound 53132.0
checking 53129 of 53132.0
at 5420 84370830 primes from 1023101357 to 2823101783
prefix length 51961884 min 1.73:3 max 31985.95:1023101273
extension length 84370830 min 31985.96:1023101357 max 53132.87:2823101783
combined length 136332714 min 1.73:3 max 53132.87:2823101783


In [15]:
len(smallprimesB), smallprimesB[-1], smallprimesB[51961882:51961888]

(136332714,
 np.int64(2823101783),
 array([1023101269, 1023101273, 1023101357, 1023101363, 1023101377,
        1023101383]))

In [16]:
smallprimesC = extendblock(smallprimesB, 900000000)

res0 2275 i 1137 x0 2823101785 block 900000255 pbound 67993.0
checking 67987 of 67993.0
at 6772 81716615 primes from 2823101791 to 4623102269
prefix length 136332714 min 1.73:3 max 53132.87:2823101783
extension length 81716615 min 53132.87:2823101791 max 67993.40:4623102269
combined length 218049329 min 1.73:3 max 67993.40:4623102269


In [17]:
len(smallprimesC), smallprimesC[-1]

(218049329, np.int64(4623102269))

In [18]:
# np.save('primesE9_46.np', smallprimesC)

### Note: 2 Jun 2025 - constellations toward end of small primes
We would like our samples of primes to include complete intervals of survival $\Delta H(p)$.  Let $q$ be the next prime above $p$,
and let $g=q-p$ be the gap between them.

Then $\Delta H(p) = [p^2,\; q^2]$ with length
$$| \Delta H(p)| \; = \; q^2-p^2 \; = \; g \cdot (2p+g) \approx 2gp$$
When $g << p$, the interval $\Delta H(p)$ has length approximately proportional to $g$.

Assuming the average gap size within $\Delta H(p)$ does not vary significantly for nearby primes, the number of primes within adjacent
intervals of survival will also be approximately proportional to $g$.  In $\Delta H(p)$ we would expect the number of primes
to be roughly
$$ \frac{q^2-p^2}{\mu} \; \approx \; \frac{g \cdot (2p+g)}{\mu}$$.

For the current array <i>smallprimes</i>, toward the largest of these primes, $q=1023094327$ and $\mu \approx 36.95036$.

The gap $g=q-p$ is an even number, and for the smallest value $g=2$ we expect $\Delta H(p)$ in this range to contain over 110 million
primes.

### An approach
1. Pick a range for small prime $p$ for which we want to obtain a sample $\Delta H(p)$.  <i>Example:</i> if we want a sample around $3.6E15$, we need to work with small primes $p \approx 6E7$.
2. Find a small prime $p$ in this range and get its index in the array <i>smallprimes[]</i>.
3. Calculate a range of gaps around $p$
4. Look for manageable gap sizes for which to create $\Delta H(p_0)$.  Remember that the expected number of gaps in $\Delta H(p_0)$ is $\frac{g \cdot (2p_0+g)}{\mu}$

In [48]:
smallprimes[-102:]-smallprimes[-103:-1]

array([ 2, 28, 72, 50, 16,  2, 12, 10, 20, 30,  6, 10, 12, 42,  6, 20, 10,
       12,  2,  6, 10, 38,  6, 24, 12, 10,  8, 10, 24, 12, 14, 16,  6, 14,
       58,  2, 28, 30,  8, 22, 20, 30, 10, 18, 14,  4,  6, 20, 18, 42, 36,
       24,  4,  2, 18, 36,  4,  2, 34, 18,  8, 22,  8, 18,  4,  8, 12, 16,
       26,  4, 24,  2, 40,  6,  2, 18,  4, 24,  2, 82, 30, 18,  8, 22, 30,
       20, 12, 10,  6,  2, 10,  8,  6, 58, 18,  2,  6,  6, 16, 54, 24, 20])

## File naming convention
To organize the files of the saved arrays of primes we use the following naming convention.

Each file begins with the prefix <i>primeblock</i> followed by a reverse scientific notation, the <i>E</i> value (the exponent)
followed by an underscore and then the first three digits of the coefficient, for the largest prime in the array.

In [20]:
# SAVE this data for use in other modules
# we comment out these lines to prevent accidental clobbering of the existing files
# np.save('primeblockEXX_nnn.npy',ptrial)

## OLDER VERSIONS OF CODE BELOW - 